In [1]:
pip install scikit-image

  Obtaining dependency information for scikit-image from https://files.pythonhosted.org/packages/ce/d0/a3f60c9f57ed295b3076e4acdb29a37bbd8823452562ab2ad51b03d6f377/scikit_image-0.22.0-cp311-cp311-win_amd64.whl.metadata
  Using cached scikit_image-0.22.0-cp311-cp311-win_amd64.whl.metadata (13 kB)
  Obtaining dependency information for networkx>=2.8 from https://files.pythonhosted.org/packages/d5/f0/8fbc882ca80cf077f1b246c0e3c3465f7f415439bdea6b899f6b19f61f70/networkx-3.2.1-py3-none-any.whl.metadata
  Using cached networkx-3.2.1-py3-none-any.whl.metadata (5.2 kB)
  Obtaining dependency information for imageio>=2.27 from https://files.pythonhosted.org/packages/02/25/66533a8390e3763cf8254dee143dbf8a830391ea60d2762512ba7f9ddfbe/imageio-2.34.0-py3-none-any.whl.metadata
  Obtaining dependency information for tifffile>=2022.8.12 from https://files.pythonhosted.org/packages/cd/0b/33610b4d0d1bb83a6bfd20ed838f52e02a44e9b439116cd4f3d424e81a80/tifffile-2024.2.12-py3-none-any.whl.metadata
  Obtainin


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: C:\Users\Basuru Yasaruwan\Contacts\Desktop\IPCV Practicals\Practical 03\IM02\Scripts\python.exe -m pip install --upgrade pip


In [10]:
pip install opencv-python==4.7.0.68

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: C:\Users\Basuru Yasaruwan\Contacts\Desktop\IPCV Practicals\Practical 03\IM02\Scripts\python.exe -m pip install --upgrade pip


In [6]:
pip list

Package                      Version
---------------------------- ---------------
absl-py                      2.1.0
anyio                        4.2.0
argon2-cffi                  23.1.0
argon2-cffi-bindings         21.2.0
arrow                        1.3.0
asttokens                    2.4.1
astunparse                   1.6.3
async-lru                    2.0.4
attrs                        23.2.0
Babel                        2.14.0
beautifulsoup4               4.12.3
bleach                       6.1.0
cachetools                   5.3.2
certifi                      2024.2.2
cffi                         1.16.0
charset-normalizer           3.3.2
colorama                     0.4.6
comm                         0.2.1
contourpy                    1.2.0
cycler                       0.12.1
debugpy                      1.8.0
decorator                    5.1.1
defusedxml                   0.7.1
executing                    2.0.1
fastjsonschema               2.19.1
fileupload                   0.1


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: C:\Users\Basuru Yasaruwan\Contacts\Desktop\IPCV Practicals\Practical 03\IM02\Scripts\python.exe -m pip install --upgrade pip


In [7]:
def apply_post_processing(prediction):
    # Convert the prediction to a binary mask using a threshold.
    # Convert the continuos values in the prediction into discrete 1s or 0s.
    # If prediction thresh is more than 0.3 it`s considered as 1 (True).
    # Otherwise a 0 (false)
    binary_mask = (prediction > 0.3).astype(np.uint8)

    if np.any(binary_mask):  # Check if the mask is not empty
        # Ensure that the binary mask is a NumPy array
        binary_mask = np.asarray(binary_mask, dtype=np.uint8)

        # Erosion operation to remove small false positives.
        # Iteration represents, how many time you moved the default kernal
        # across the image matrix. Erosion operation will reduce the noise
        # and let the important feature emerged more strongly.
        eroded_mask = cv2.erode(binary_mask, erosion_kernel, iterations=8)

        # Dilation operation to restore the size of the image 
        #and readjust the too much impacted regions during the erosion process
        post_processed_mask = cv2.dilate(eroded_mask, erosion_kernel, iterations=5)

        return post_processed_mask
    else:
        return binary_mask

In [13]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from scipy.ndimage import binary_erosion, binary_dilation

# Load your pre-trained VGG16 model for gun holding
vgg_model = load_model('custom_gun_model.h5')

# Create a VideoCapture object
cap = cv2.VideoCapture(0)

# Set the width and height of the video capture
# 3- means width. Set the pixel density for width.
cap.set(3, 640)
# 4- means height. Set the pixel density for height.
cap.set(4, 480)

# Initialize variables for temporal smoothing
history_length = 5
prediction_history = []

# Initialize a binary erosion kernel with size of 5 x 5. This is a smoothing
# technique used to reduce the noise, as averages the pixel in the overlapping
# region of the kernel.
erosion_kernel = np.ones((5, 5), np.uint8)

while True:
    # Capture a frame from the video feed
    ret, frame = cap.read()

    # Check if the frame is not empty
    if not ret:
        print("Error: Could not read frame")
        break

    # Preprocess the frame for VGG16 model
    frame_vgg = cv2.resize(frame, (224, 224))
    img_array = image.img_to_array(frame_vgg)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Fine-tune the model on your specific gun detection task.
    # 2 zeros are referring to like 2 dimentional representation of the
    # prediction
    prediction = vgg_model.predict(img_array)[0][0]

    # Append the current prediction to the history
    prediction_history.append(prediction)

    # Keep only the last 'history_length' predictions
    if len(prediction_history) > history_length:
        prediction_history = prediction_history[1:]
        
    # camera feed is a live feed subject to variations. History prediction
    # is maintained as a moving average strategy to overcome the variations
    # which could happen in the camera`s live feed and to make the prediction
    # more stabalized.

    # Calculate the smoothed prediction using a simple moving average
    smoothed_prediction = np.mean(prediction_history)

    # Apply post-processing to the prediction
    post_processed_prediction = apply_post_processing(smoothed_prediction)

    # Assuming binary classification (gun or not)
    prediction_label = "Gun Detected" if post_processed_prediction > 0.5 else "No Gun Detected"

    # Display the frame with smoothed prediction
    cv2.putText(frame, f"Prediction: {prediction_label}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    cv2.imshow('Gun Detection', frame)
    #cv2.waitKey(0)

    # Break the loop when 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the VideoCapture and close all windows
cap.release()
cv2.destroyAllWindows()



1/1 [==============================] - 0s 94ms/step
